In [ ]:
# Installazione pacchetti (da eseguire solo la prima volta)
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q qwen-omni-utils -U
!pip install -q bitsandbytes -U
!pip install -q accelerate
!pip install -q pandas # pandas for CSV operations, though not strictly needed here for basic csv

# Import
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import re
import gc
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"
# --- Preparazione della lista dei prompt dal CSV ---
# Percorso del file CSV
csv_file = "/kaggle/input/qwennn/analisi_audio_intera_cartella_con_features (7).csv" #
# Lista per contenere tutti i prompt generati
prompt_list = []
# Lista per salvare i dati per il CSV dei risultati finali
results_for_csv = []

# Dizionario temporaneo per raggruppare le descrizioni per canzone
current_song_data = {}

try:
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header

        for row in reader: #
            song_name_cell = row[0].strip() # Nome Canzone
            description_text = row[4].strip() # Colonna 'Descrizione' (indice 4 basato sull'immagine bf21ef.png)
            description_type = row[3].strip() # Colonna 'Tipo Descrizione' (indice 3)

            # Se la cella Nome Canzone non è vuota, significa che è l'inizio di una nuova canzone
            if song_name_cell:
                # Se c'era una canzone precedente da elaborare, la aggiungiamo alla prompt_list
                if current_song_data:
                    # Assicurati di avere esattamente 6 descrizioni
                    # Dovresti decidere l'ordine (es. prima i frammenti, poi il totale)
                    # Qui assumo che le descrizioni vengano raccolte nell'ordine in cui appaiono nel CSV
                    # e che la "Descrizione Totale" sia l'ultima.
                    ordered_descriptions = []
                    # Raccogli i frammenti
                    for i in range(1, 6): # Assumendo Frammento 1 a 5, come da immagine cc456f.png
                        ordered_descriptions.append(current_song_data.get(f'Frammento {i}', ''))
                    
                    # Raccogli la descrizione totale
                    overall_desc_found = False
                    for desc_type_key in ['Totale', 'Descrizione Totale', 'Overall Description']: # Controlla vari nomi possibili
                        if desc_type_key in current_song_data:
                            ordered_descriptions.append(current_song_data[desc_type_key])
                            overall_desc_found = True
                            break
                    if not overall_desc_found:
                        ordered_descriptions.append('') # Aggiungi una stringa vuota se la descrizione totale manca

                    # Rimuovi eventuali stringhe vuote in eccesso se ci sono meno di 6 descrizioni
                    ordered_descriptions = [d for d in ordered_descriptions if d.strip()]
                    while len(ordered_descriptions) < 6:
                        ordered_descriptions.append('') # Assicurati che ci siano sempre 6 descrizioni

                    # Ora genera il prompt per la canzone precedente
                    prompt = f"Nome Canzone: {current_song_data['Nome Canzone']}\n\n"
                    prompt += "Dati questi frammenti descrittivi di una canzone, valuta su una scala da 0 a 255 quanto ciascuno dei seguenti stati emotivi è evocato dalla descrizione:\n"
                    prompt += "agitazione/movimento (R), calma (G), tristezza (B).\n\n"
                    prompt += "Restituisci **ESATTAMENTE 6 triple RGB COMPLETE**, una dopo l'altra, su una SINGOLA riga. \n"
                    prompt += "Non includere nessuna spiegazione, numerazione, elenchi puntati o altro testo aggiuntivo. Non usare la modalità di pensiero ('thinking mode').\n"
                    prompt += "Inizia la riga con 'RGB:' e poi le 6 triple. Ogni tripla deve contenere esattamente tre valori separati da virgole.\n\n"
                    prompt += "Ecco l'ESEMPIO ESATTO del formato desiderato (solo 6 triple complete, su una riga):\n"
                    prompt += "RGB: (R1, G1, B1) (R2, G2, B2) (R3, G3, B3) (R4, G4, B4) (R5, G5, B5) (R6, G6, B6)\n\n"
                    prompt += "Ecco i frammenti:\n"

                    ordered_prompts_content = []
                    # Mappa il tipo di descrizione all'etichetta desiderata e un ordine numerico
                    desc_order = {f'Frammento {j}': j for j in range(1, 6)} # Da Frammento 1 a Frammento 5
                    desc_order['Totale'] = 6 # La descrizione totale è l'ultima

                    temp_descs = []
                    for k, v in current_song_data.items():
                        if k in desc_order:
                            temp_descs.append({'type': k, 'content': v, 'order': desc_order[k]})
                    temp_descs.sort(key=lambda x: x['order'])

                    for d in temp_descs:
                        label = d['type']
                        if label == 'Totale':
                            label = "Descrizione Totale"
                        ordered_prompts_content.append(f"- {label}: {d['content']}\n")

                    prompt += "".join(ordered_prompts_content)

                    prompt_list.append({'song_name': current_song_data['Nome Canzone'], 'prompt_text': prompt})
                
                # Inizia una nuova canzone
                current_song_data = {
                    'Nome Canzone': song_name_cell
                }
            
            # Aggiungi la descrizione al dizionario della canzone corrente
            current_song_data[description_type] = description_text #

        # Non dimenticare di aggiungere l'ultima canzone dopo la fine del loop
        if current_song_data:
            ordered_descriptions = []
            for i in range(1, 6): # Frammento 1 a 5
                ordered_descriptions.append(current_song_data.get(f'Frammento {i}', ''))
            
            overall_desc_found = False
            for desc_type_key in ['Totale', 'Descrizione Totale', 'Overall Description']:
                if desc_type_key in current_song_data:
                    ordered_descriptions.append(current_song_data[desc_type_key])
                    overall_desc_found = True
                    break
            if not overall_desc_found:
                ordered_descriptions.append('')

            # Assicurati che ci siano esattamente 6 descrizioni, aggiungendo stringhe vuote se necessario
            ordered_descriptions = [d for d in ordered_descriptions if d.strip()] # Pulisci prima
            while len(ordered_descriptions) < 6:
                ordered_descriptions.append('')

            prompt = f"Nome Canzone: {current_song_data['Nome Canzone']}\n\n"
            prompt += "Dati questi frammenti descrittivi di una canzone, valuta su una scala da 0 a 255 quanto ciascuno dei seguenti stati emotivi è evocato dalla descrizione:\n"
            prompt += "agitazione/movimento (R), calma (G), tristezza (B).\n\n"
            prompt += "Restituisci **ESATTAMENTE 6 triple RGB COMPLETE**, una dopo l'altra, su una SINGOLA riga. \n"
            prompt += "Non includere nessuna spiegazione, numerazione, elenchi puntati o altro testo aggiuntivo. Non usare la modalità di pensiero ('thinking mode').\n"
            prompt += "Inizia la riga con 'RGB:' e poi le 6 triple. Ogni tripla deve contenere esattamente tre valori separati da virgole.\n\n"
            prompt += "Ecco l'ESEMPIO ESATTO del formato desiderato (solo 6 triple complete, su una riga):\n"
            prompt += "RGB: (R1, G1, B1) (R2, G2, B2) (R3, G3, B3) (R4, G4, B4) (R5, G5, B5) (R6, G6, B6)\n\n"
            prompt += "Ecco i frammenti:\n"

            ordered_prompts_content = []
            desc_order = {f'Frammento {j}': j for j in range(1, 6)}
            desc_order['Totale'] = 6 

            temp_descs = []
            for k, v in current_song_data.items():
                if k in desc_order:
                    temp_descs.append({'type': k, 'content': v, 'order': desc_order[k]})
            temp_descs.sort(key=lambda x: x['order'])

            for d in temp_descs:
                label = d['type']
                if label == 'Totale':
                    label = "Descrizione Totale"
                ordered_prompts_content.append(f"- {label}: {d['content']}\n")

            prompt += "".join(ordered_prompts_content)

            prompt_list.append({'song_name': current_song_data['Nome Canzone'], 'prompt_text': prompt})

except FileNotFoundError:
    print(f"Errore: Il file CSV '{csv_file}' non è stato trovato.")
    print("Assicurati che il file si trovi nel percorso specificato.")
    exit()

# --- Fine preparazione della lista dei prompt ---


# Parametri modello

compute_dtype = torch.float16

# Configurazione quantizzazione 4-bit
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype
)

# Caricamento tokenizer e modello
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quant_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    torch_dtype=compute_dtype
)

# Loop sui prompt
for idx, entry in enumerate(prompt_list, start=1):
    song_name = entry['song_name']
    prompt_text = entry['prompt_text']

    print(f"\n=== Elaborazione canzone {idx}: {song_name} ===\n")

    chat = [{"role": "user", "content": prompt_text}]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, enable_thinking=False)

    inputs = tokenizer(chat_text, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.1,
        num_return_sequences=1,
    )

    response_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    print("Risposta generata dal modello:\n")
    print(response_text)

    # --- MODIFICA REGEX ---
    # Questa espressione regolare cerca qualsiasi sequenza di (numero, numero, numero)
    # catturando i tre numeri.
    rgb_value_pattern = re.compile(r"\((\d{1,3}),\s*(\d{1,3}),\s*(\d{1,3})\)")
    found_rgb_matches = rgb_value_pattern.findall(response_text)
    # --- FINE MODIFICA ---

    current_song_rgb_values = [song_name] # Inizia con il nome della canzone

    # Popola fino a 6 valori RGB. Se ne trova meno, i rimanenti saranno (0,0,0).
    for i in range(6): # Ci aspettiamo esattamente 6 RGB
        if i < len(found_rgb_matches):
            try:
                # Accedi ai gruppi catturati direttamente dalla tupla
                r, g, b = int(found_rgb_matches[i][0]), int(found_rgb_matches[i][1]), int(found_rgb_matches[i][2])
                r, g, b = max(0, min(255, r)), max(0, min(255, g)), max(0, min(255, b))
                current_song_rgb_values.append(f"rgb({r}, {g}, {b})")
            except ValueError:
                # Questo gestirà anche casi come (200) dove manca il resto dei numeri
                print(f"Errore nel parsing RGB per il match {found_rgb_matches[i] if i < len(found_rgb_matches) else 'inatteso'}. Usando nero.")
                current_song_rgb_values.append("rgb(0, 0, 0)")
        else:
            # Se non trova abbastanza RGB, aggiunge il nero
            current_song_rgb_values.append("rgb(0, 0, 0)")
            print(f"Avviso: Meno di 6 triple RGB complete trovate per {song_name}. Aggiungendo nero per il frammento {i+1}.")

    results_for_csv.append(current_song_rgb_values)

    # Pulisci la memoria GPU e CPU dopo ogni elaborazione
    del inputs, outputs # <--- 'outputs' è definito qui, all'interno del loop
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

# --- Salvataggio finale dei dati ---

# Saving RGBs only to a separate CSV file
print("\n--- Salvataggio dei colori RGB in CSV ---")
output_csv_file = "valutazione_rgb_segmenti.csv"
csv_header = ['Nome Canzone', 'Frammento 1', 'Frammento 2', 'Frammento 3', 'Frammento 4', 'Frammento 5', 'Descrizione Totale']

with open(output_csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(csv_header)
    writer.writerows(results_for_csv)
print(f"Colori RGB salvati in '{output_csv_file}'")

print("\nProcessing complete. No visualizations or JSON output were generated as per your request.")
print("The RGB values are saved in the CSV file.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 52.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00:00:0100:01


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...


=== Elaborazione canzone 20: 000255.mp3 ===

Risposta generata dal modello:

RGB: (255, 100, 30) (255, 150, 30) (128, 128, 128) (255, 180, 30) (255, 120, 30) (255, 140, 30)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione canzone 21: 000256.mp3 ===

Risposta generata dal modello:

RGB: (255, 30, 30) (255, 30, 30) (255, 30, 30) (255, 30, 30) (255, 30, 30) (255, 30, 30)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione canzone 22: 000368.mp3 ===

